In [6]:
import pandas as pd
from Levenshtein import distance
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import re
from src.utils import extract_quintuplet

In [31]:
df = pd.read_csv('../Data/test_data.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   aoriginal_id                          200 non-null    int64 
 1   content                               200 non-null    object
 2   final_sentiment                       200 non-null    object
 3   baseline_aspect_category              195 non-null    object
 4   tweet_type                            200 non-null    object
 5   clean_tweet                           200 non-null    object
 6   label                                 200 non-null    object
 7   corrected_label                       25 non-null     object
 8   keterangan                            8 non-null      object
 9   quintuplet_label                      200 non-null    object
 10  postprocess_quintuplet                200 non-null    object
 11  p21_model_prediction            

In [33]:
df = df.dropna(subset=['baseline_aspect_category'])

# Label diff

In [39]:
pred_quadruplet_col = 'p21_model_prediction'
ori_quadruplet_col = 'postprocess_quintuplet'

In [40]:
aspect_categories = ['price', 'produk', 'payment', 'website&apps', 'delivery', 'customerservice', 'user']
annot_errors = []
model_errors = []
df['annotated_multilabel'] = ''
df['keyword_multilabel'] = ''
df['model_multilabel'] = ''
for i in range(len(df)):
    row = df.iloc[i]
    #for annotated aspect categories
    annotated_quintuplet = extract_quintuplet(row[ori_quadruplet_col])
    annotated_aspect = []
    #for model aspect categories
    model_quintuplet = extract_quintuplet(row[pred_quadruplet_col])
    model_aspect = []
    #for annotated quadruplet
    for quintuplet in annotated_quintuplet:
        #for annotated
        entities, aspect_term, opinion_term, sentiment, aspect_category = quintuplet
        annotated_aspect.append(aspect_category)
        #annotated_aspect.append(aspect_categories.index(aspect_category))
    #for model quadruplet
    for quintuplet in model_quintuplet:
        #for annotated
        entities, aspect_term, opinion_term, sentiment, aspect_category = quintuplet
        model_aspect.append(aspect_category)
        #model_aspect.append(aspect_categories.index(aspect_category))
    #for annotated label
    df.at[i, 'annotated_multilabel'] = set(annotated_aspect)
    #for model label
    df.at[i, 'model_multilabel'] = set(model_aspect)
    #for keyword label
    labels = row['baseline_aspect_category'].split(';') 
    keyword_multilabels = [label.replace(' ', '') for label in labels if label != '']
    #keyword_multilabels = [aspect_categories.index(keyword_multilabels) for label in labels]
    df.at[i, 'keyword_multilabel'] = set(keyword_multilabels)

In [41]:
df.head()[[ori_quadruplet_col, pred_quadruplet_col, 'baseline_aspect_category']]

,postprocess_quintuplet,p21_model_prediction,baseline_aspect_category
0,"(olx,_,ada manusiawinya,positive,user);(fb,_,k...","(olx,_,ada manusiawinya,positive,user);(fb,_,k...",produk;
1,"(shopee,shopee express,paling cepet,positive,d...","(shopee,shopee express,paling cepet,positive,d...",delivery; produk;
2,"(shopee,cs shopee,bodong,negative,customerserv...","(shopee,cs shopee,bodong,negative,customerserv...",payment; produk;
3,"(shopee,admin shopee,makin gedeee,negative,price)","(shopee,admin shopee,makin gedeee,negative,price)",produk;
4,"(shopee,pengiriman instan,bener-bener instan b...","(shopee,pengiriman instan,bener-bener instan b...",price; produk;


In [42]:
df.head()[['annotated_multilabel', 'model_multilabel', 'keyword_multilabel']]

,annotated_multilabel,model_multilabel,keyword_multilabel
0,{user},{user},{produk}
1,{delivery},{delivery},"{produk, delivery}"
2,{customerservice},{customerservice},"{produk, payment}"
3,{price},{price},{produk}
4,{delivery},{delivery},"{produk, price}"


In [43]:
y_true = MultiLabelBinarizer(classes=aspect_categories).fit_transform(df['annotated_multilabel'])
y_keyword = MultiLabelBinarizer(classes=aspect_categories).fit_transform(df['keyword_multilabel'])
y_model = MultiLabelBinarizer(classes=aspect_categories).fit_transform(df['model_multilabel'])

In [44]:
#for keyword
print(classification_report(y_true,y_keyword, target_names=aspect_categories))

                 precision    recall  f1-score   support

          price       0.33      0.33      0.33        30
         produk       0.09      1.00      0.16        17
        payment       0.25      0.22      0.24        18
   website&apps       0.33      0.09      0.15        43
       delivery       0.56      0.34      0.42        68
customerservice       0.30      0.25      0.27        12
           user       0.00      0.00      0.00        13

      micro avg       0.20      0.30      0.24       201
      macro avg       0.27      0.32      0.22       201
   weighted avg       0.36      0.30      0.27       201
    samples avg       0.16      0.29      0.20       201



C:\Users\Danendra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danendra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danendra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classificatio

In [45]:
#for model
print(classification_report(y_true, y_model, target_names=aspect_categories))

                 precision    recall  f1-score   support

          price       0.97      1.00      0.98        30
         produk       1.00      1.00      1.00        17
        payment       1.00      1.00      1.00        18
   website&apps       1.00      1.00      1.00        43
       delivery       1.00      0.99      0.99        68
customerservice       1.00      0.83      0.91        12
           user       1.00      0.92      0.96        13

      micro avg       0.99      0.98      0.99       201
      macro avg       1.00      0.96      0.98       201
   weighted avg       1.00      0.98      0.99       201
    samples avg       0.97      0.96      0.97       201



C:\Users\Danendra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danendra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
'produk' in df.iloc[0]['keyword_multilabel']

True

In [49]:
for i in range(len(df)):
    row = df.iloc[i]
    if 'delivery' in row['model_multilabel'] and 'delivery' not in row['keyword_multilabel']:
        print(i)


4
7
14
16
28
41
44
47
50
53
56
58
64
75
79
80
82
87
92
100
104
109
115
120
126
128
131
137
138
142
148
153
155
156
157
158
162
164
165
178
180
183
185
187


# Sentiment

In [26]:
df.head(1)

,aoriginal_id,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,label,corrected_label,keterangan,quintuplet_label,postprocess_quintuplet,p21_model_prediction,p00_model_prediction,postprocessed_p21_prediction,postprocessed_postprocess_quintuplet,postprocessed_p00_prediction,annotated_multilabel,keyword_multilabel,model_multilabel
0,1.647222e+18,@sosmedkeras mendingan di olx masih ada manusi...,neutral,produk;,SUBJECTIVE,Mendingan di olx masih ada manusiawinya sedik...,"(olx, _, ada manusiawinya, positive, user);(fb...",NaN,NaN,"(olx, _, ada manusiawinya, positive, user);(fb...","(olx,_,ada manusiawinya,positive,user);(fb,_,k...","(olx,_,ada manusiawinya,positive,user);(fb,_,k...","(olx,_,ada manusiawinya,positive,user);(fb,_,k...","(olx,_,ada manusiawinya,positive,user);(fb,_,k...","(olx,_,ada manusiawinya,positive,user);(fb,_,k...","(olx,_,ada manusiawinya,positive,user);(fb,_,k...",{user},{produk},{user}


In [30]:
row[ori_quadruplet_col]

nan

In [34]:
annot_sents = []
keyword_sents = []
model_sents = []

ori_quadruplet_col = 'postprocess_quintuplet'
pred_quadruplet_col = 'p21_model_prediction'
keyword_sentiment_col = 'final_sentiment'
for i in range(len(df)):
    row = df.iloc[i]
    #for annotated aspect categories
    annotated_quadruplets = extract_quintuplet(row[ori_quadruplet_col])
    annot_sent = []
    #for model aspect categories
    model_quadruplets = extract_quintuplet(row[pred_quadruplet_col])
    model_sent = []
    #for annotated quadruplet
    for quadruplet in annotated_quadruplets:
        #for annotated
        entity, aspect_term, opinion_term, sentiment, aspect_category = quadruplet
        annot_sent.append(sentiment)
    #for model quadruplet
    for quadruplet in model_quadruplets:
        #for annotated
        entity, aspect_term, opinion_term, sentiment, aspect_category = quadruplet
        model_sent.append(sentiment)
    annot_sent = set(annot_sent)
    model_sent = set(model_sent)
    if len(annot_sent)>1 or len(model_sent)>1:
        continue
    else:
        annot_sents.append(list(annot_sent)[0])
        model_sents.append(list(model_sent)[0])
        keyword_sents.append(row[keyword_sentiment_col])

In [35]:
len(keyword_sents), len(model_sents), len(annot_sents), df.shape[0]

(186, 186, 186, 195)

In [36]:
print(classification_report(y_true=annot_sents, y_pred=keyword_sents))

              precision    recall  f1-score   support

    negative       0.79      0.88      0.83       130
     neutral       0.09      0.50      0.16         6
    positive       0.80      0.16      0.27        50

    accuracy                           0.67       186
   macro avg       0.56      0.51      0.42       186
weighted avg       0.77      0.67      0.66       186



In [37]:
print(classification_report(y_true=annot_sents, y_pred=model_sents))

              precision    recall  f1-score   support

    negative       0.99      0.99      0.99       130
     neutral       1.00      1.00      1.00         6
    positive       0.98      0.98      0.98        50

    accuracy                           0.99       186
   macro avg       0.99      0.99      0.99       186
weighted avg       0.99      0.99      0.99       186

